In [ ]:
!pip install -q flatbuffers==2.0.0
!pip install -q mediapipe==0.9.0
!pip install -q mediapipe-model-maker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.6/33.6 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 98.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.1/175.1 kB 18.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wh

In [ ]:
import os

NUM_EXAMPLES = 5
IMAGES_PATH = "drive/Shareddrives/KPZ 2023 - rozpoznawanie języka migowego/data/prepared"

# Get the list of labels from the list of folder names.
labels = []
for i in os.listdir(IMAGES_PATH):
  if os.path.isdir(os.path.join(IMAGES_PATH, i)):
    labels.append(i)

print(labels)

['CZ', 'A', 'B', 'C', 'D', 'E', 'None', 'F', 'G', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'W', 'Y', 'Z', 'SZ', 'H']


In [ ]:
import matplotlib.pyplot as plt

# plt.close()

# Show the images.
for label in labels:
  if label == 'None': continue

  label_dir = os.path.join(IMAGES_PATH, label)
  example_filenames = os.listdir(label_dir)[:NUM_EXAMPLES]
  fig, axs = plt.subplots(1, NUM_EXAMPLES, figsize=(10,2))
  for i in range(NUM_EXAMPLES):
    axs[i].imshow(plt.imread(os.path.join(label_dir, example_filenames[i])))
    axs[i].get_xaxis().set_visible(False)
    axs[i].get_yaxis().set_visible(False)
  fig.suptitle(f'Showing {NUM_EXAMPLES} examples for {label}')

plt.show()

In [ ]:
# Importing model maker library
from mediapipe_model_maker import gesture_recognizer

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
print("Loading the sign-language image archive...")
print("This takes a while")
data = gesture_recognizer.Dataset.from_folder(
    dirname=IMAGES_PATH,
    hparams=gesture_recognizer.HandDataPreprocessingParams(
        shuffle=True,
        min_detection_confidence=0.75
    )
)

Loading the sign-language image archive...
This takes a while


In [ ]:
print("Splitting the archive into training, validation and test dataset...")
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

Splitting the archive into training, validation and test dataset...


In [ ]:
print("Creating learning parameters")
hparams = gesture_recognizer.HParams(
    export_dir="pjm_recognizer_model",
    batch_size=3,
    epochs=15,
    gamma=2
)
print("Creating model options")
model_options = gesture_recognizer.ModelOptions(
    dropout_rate=0.05 # hidden layers
)
print("Creating gesture recognizer options")
options = gesture_recognizer.GestureRecognizerOptions(
    hparams=hparams,
    model_options=model_options
)

print("Creating gesture recognizer")
model = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

Creating learning parameters
Creating model options
Creating gesture recognizer options
Creating gesture recognizer
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hand_embedding (InputLayer)  [(None, 128)]            0         
                                                                 
 batch_normalization (BatchN  (None, 128)              512       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 128)               0         
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 custom_gesture_recognizer_o  (None, 25)               3225      
 ut (Dense)                                                      
           

In [ ]:
loss, acc = model.evaluate(test_data, batch_size=3)
print(f"Test loss:{loss}, Test accuracy:{acc}")
model.summary()

73/73 [==============================] - 2s 6ms/step - loss: 0.3467 - categorical_accuracy: 0.8111
Test loss:0.3467472493648529, Test accuracy:0.8110598921775818
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hand_embedding (InputLayer)  [(None, 128)]            0         
                                                                 
 batch_normalization (BatchN  (None, 128)              512       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 128)               0         
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 custom_gesture_recognizer_o  (None, 25)               3225      
 ut (Dense)                    

In [ ]:
model.export_model()

In [ ]:
!mv pjm_recognizer_model/gesture_recognizer.task pjm_v3.task